In [11]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')
pd.options.display.float_format = '{:,.0f}'.format

In [28]:
d1 = pd.read_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/blockchain-dashboards/02_NFT/Raw_Data/01-25.csv', on_bad_lines='skip')
d2 = pd.read_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/blockchain-dashboards/02_NFT/Raw_Data/02-25.csv', on_bad_lines='skip')
d3 = pd.read_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/blockchain-dashboards/02_NFT/Raw_Data/03-25.csv', on_bad_lines='skip')
d4 = pd.read_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/blockchain-dashboards/02_NFT/Raw_Data/04-25.csv', on_bad_lines='skip')

df = pd.concat([d1, d2, d3,d4], ignore_index=True)
df.head()

,BLOCK_TIMESTAMP,SELLER_ADDRESS,BUYER_ADDRESS,QUANTITY,CURRENCY_SYMBOL,PRICE,PRICE_USD,ORIGIN_FROM_ADDRESS,ORIGIN_TO_ADDRESS,PROJECT_NAME,TOKEN_ID,TOKEN_STANDARD
0,2025-01-05 02:44:23.000,0x5255ba61aa23d11859d219771201fc8a85c8e9e9,0xbff5abac0357cf8987f453eb15b111c5089cd25e,1,ETH,0,13,0xbff5abac0357cf8987f453eb15b111c5089cd25e,0x0000000000000068f116a894984e2db1123eb395,SILENCE,2,erc1155
1,2025-01-05 02:40:47.000,0x3fbdf3e4ad2c661c8ec6067a69e339b1a730eb52,0x3edb011de8cecd5f8dc0d269e79e0eb33dc5b58c,1,ETH,1,"4,525",0x3edb011de8cecd5f8dc0d269e79e0eb33dc5b58c,0x0000000000000068f116a894984e2db1123eb395,CryptoDickbutts S3,1084,erc721
2,2025-01-05 03:57:59.000,0xcca4446b31a94d7708729a13bea16e73d47e1eed,0x880db1abee87651b667615c24db3a30aa6aaa716,1,ETH,0,188,0x880db1abee87651b667615c24db3a30aa6aaa716,0x0000000000000068f116a894984e2db1123eb395,HV-MTL,9523,erc721
3,2025-01-05 04:01:35.000,0xd01cc179fb530138fbb9c957f08301fa78c3bccc,0x8da8e254ae211c60aa0deba314ec2b11d0d4a50f,1,ETH,0,32,0x8da8e254ae211c60aa0deba314ec2b11d0d4a50f,0x0000000000000068f116a894984e2db1123eb395,Fang Gang,5550,erc721
4,2025-01-05 03:34:11.000,0x64241be80ee10fa0a2dff64267f6f8ac7b384fe8,0x25b1a05210cd81317e2addf657b7d5e4b45fdf84,1,ETH,0,71,0x25b1a05210cd81317e2addf657b7d5e4b45fdf84,0x0000000000000068f116a894984e2db1123eb395,Pudgy Milady,4457,erc721


# Data Manupulation

In [39]:
# map category

category_mapping = {
    'Nobody': 'PFP',
    'PUNKBITS': 'PFP',
    'Elemental': 'Game',
    'LilPudgys': 'PFP',
    'Inka Blinks': 'Art',
    'Hen House': 'Game',
    'INSC Plus': 'Collectibles',
    'Persona': 'PFP',
    'TinFun': 'Collectibles',
    'Pillheads': 'Art',
    'Sheboshis': 'Collectibles',
    'THNFTees: Dorks': 'Collectibles',
    'Moonbirds': 'PFP',
    'Puppet Milady': 'PFP',
    'Mint Genesis NFT': 'Utility',
    'Rosentica: Starfall Travelers': 'Collectibles',
    'Steamboat Willie': 'IP',
    'EigenWorlds': 'Game',
    'MutantApeYachtClub': 'PFP',
    'EigenLayer World Builders': 'Game',
    "LVCIDIA// RESOURCES": "Metaverse",
    "NPC MFT": "Game",
    "cirth {ferc-721}": "Art",
    "TenNeighbors": "PFP",
    "Pudgy Milady": "PFP",
    "King2d": "PFP",
    "Ethereum Puppets": "Collectibles",
    "Early Retired Penguins Club": "PFP",
    "Age of Dino - Dinosty": "Game",
    "Zokio": "Art",
    "The Buns": "PFP",
    "TENJIN": "PFP",
    "unused_1": "Utility",
    "Azuki": "PFP",
    "parallel": "Game",
    "Steamboat Willie Public Domain 2024": "IP",
    "buildr.build": "Utility",
    "Something": "Art",
    "CLOAKS": "PFP",
    "Webverse": "Metaverse",
    "Cece Cube": "Collectibles",
    "Honey Comb": "Collectibles",
    "Steamboat Willie's Riverboat": "IP",
    "OpenSea Shared Storefront": "Utility",
    "XocietyFrontier": "Game",
    "Ethereum Peppets": "Collectibles",
    "Shishi": "Art",
    "Art Blocks": "Art",
    "MATR1X KUKU": "Game",
    "KillaCubs": "PFP",
    "Punk Armada": "PFP",
    "Gemesis": "Collectibles",
    "Early Retired Cats Club": "PFP",
    "y00ts": "PFP",
    "Citizen Zero": "Game",
    "Dnner": "PFP",
    "Mavia Land": "Land",
    "Nakamigos": "PFP",
    "The Warlords of Champions Tactics": "Game",
    "CALIVERSE Pioneer": "Metaverse",
    "DeGods": "PFP",
    "THE WHITE RABBIT": "Art",
    "Rough Ryders": "PFP",
    "Lasogette NFT": "Art",
    "TheCrypt": "Game",
    "CloneX": "PFP",
    "Farm Land by Pixels": "Land",
    "Uncharted LandsX Spirits": "Game",
    "The LP": "Music",
    "Mara": "PFP",
    "PXNKAP3": "PFP",
    "AOI Engine": "Game",
    "OVERWORLD INCARNA": "Game",
    "bitsCrunch Genesis Pass": "Utility",
    "Otherdeed Expanded": "Land",
    "ParallelPlanetfall": "Game",
    "Sappy Seals": "PFP",
    "Genkai": "PFP",
    "Hasbulla NFT": "Collectibles",
    "ANOMALY A.I. by Star Im": "Art",
    "Ninja Squad": "PFP",
    "TEST NFT": "Collectibles",
    "Zorbs": "Collectibles",
    "Lil Milady Pudgys": "PFP",
    "PhaseTwo": "Utility",
    "CYBERCREW [C5]": "PFP",
    "Seizon": "PFP",
    "PudgyPenguins": "PFP",
    "Froggy Friends": "PFP",
    "Opepen Edition": "Art",
    "UndeadChef": "PFP",
    "Doodles": "PFP",
    "Moonbirds: Mythics": "PFP",
    "The Ant Republic": "PFP",
    "LULU": "PFP",
    "Yakuza Pandas": "PFP",
    "Opepen Chat": "Social",
    "Emby": "PFP",
    "Shrapnel Operators Collection": "Game",
    "Faces": "Art",
    "MysteryBean": "Collectibles",
    "CrayonMilady": "PFP",
    "Milady": "PFP",
    "Poglin: Battle For Havens Destiny": "Game",
    "Pengubits": "PFP",
    "loomlocknft": "Collectibles",
    "Pixelmon": "Game",
    "PudgyPresent": "Collectibles",
    "GLHF": "Game",
    "Sofamon Seed": "Game",
    "Immortals": "Game",
    "Otherdeed": "Land",
    "SEIZON - ZENITH": "PFP",
    "Pixl Pets": "Game",
    "Tekomon": "Game",
    "Punks2023": "PFP",
    "VeeFriends Series 2": "PFP",
    "Kagami Academy": "PFP",
    "Checks": "Art",
    "Creepz by OVERLORD": "PFP",
    "Supreme Kong 2": "PFP",
    "KaijuKingz": "PFP",
    "Spinheads360": "Art",
    "Galaxia Genesis Pass": "Utility",
    "BoredApeKennelClub": "PFP",
    "Long Live The Internet": "Art",
    "Portaloftruthofficial": "PFP",
    "RGBytes": "Art",
    "VibeNauts 0x": "PFP",
    "Animalia Tundra Titans": "Game",
    "KOKODI": "PFP",
    "Cypher Dunks": "PFP",
    "Quirklings": "PFP",
    "Seekers": "Metaverse",
    "BoredApeYachtClub": "PFP",
    "Saved Souls": "PFP",
    "Cool Cats": "PFP",
    "zkLegend Genesis NFT": "Utility",
    "Project AEON": "Game",
    "Project X": "Game",
      'Rarible': 'Collectibles',
    'BitmapPunks': 'PFP',
    'Good Vibes Club': 'PFP',
    'NameWrapper': 'Domain Name',
    'The Bears': 'PFP',
    'MetaWinners': 'Collectibles',
    'The Nakamotos': 'PFP',
    'tarifftown': 'Game',
    'loose balloons by jeremy fall': 'Art',
    'KABOOM!': 'Art',
    'Yumemono ☆ 夢物': 'Art',
    'TheMerge': 'Art',
    'Jimmy': 'PFP',
    "Sandbox's ASSETs": 'Metaverse',
    'BeeOS': 'Utility',
    'SNACKGANG': 'PFP',
    'Jirasan': 'PFP',
    'Raccoons: Genesis Pass': 'Utility',
    'DXNK': 'PFP',
    'Art DeCC0s': 'Art',
    'Dooplicator': 'Utility',
    'G2': 'Collectibles',
    'Baby Burn: OpenSea Redeemable Example': 'Utility',
    'XPSR24': 'Collectibles',
    'Moriusa': 'PFP',
    'InfiniGods - GOD VAULTS': 'Game',
    'Memoria - AL': 'PFP',
    "Sandbox's LANDs": 'Land',
    '$WRLD Horses': 'Game',
    'Quills Adventure': 'Game',
    'SPCHRONICLES': 'Collectibles',
    'GNOMA': 'PFP',
    'Kaito Genesis': 'PFP',
    'Potatoz': 'PFP',
    'PrismatiPets': 'Collectibles',
    'Captainz': 'PFP',
    'HV-MTL': 'PFP',
    'Wafuku': 'PFP',
    'Genesis Box': 'Collectibles',
    'Meebits': 'PFP',
}

df['CATEGORY'] = df['PROJECT_NAME'].map(category_mapping)

df['CATEGORY'] = df['CATEGORY'].fillna('Other')

In [40]:
df['BLOCK_TIMESTAMP'] = df['BLOCK_TIMESTAMP'].astype(str).str.replace('\ufeff', '', regex=True)

df['BLOCK_TIMESTAMP'] = pd.to_datetime(df['BLOCK_TIMESTAMP'], errors='coerce')
df = df.dropna(subset=['BLOCK_TIMESTAMP'])

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 399451 entries, 0 to 399453
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   BLOCK_TIMESTAMP      399451 non-null  datetime64[ns]
 1   SELLER_ADDRESS       399451 non-null  object        
 2   BUYER_ADDRESS        399451 non-null  object        
 3   QUANTITY             399451 non-null  float64       
 4   CURRENCY_SYMBOL      399263 non-null  object        
 5   PRICE                399451 non-null  float64       
 6   PRICE_USD            399451 non-null  float64       
 7   ORIGIN_FROM_ADDRESS  399451 non-null  object        
 8   ORIGIN_TO_ADDRESS    399445 non-null  object        
 9   PROJECT_NAME         382129 non-null  object        
 10  TOKEN_ID             399451 non-null  object        
 11  TOKEN_STANDARD       399451 non-null  object        
 12  CATEGORY             399451 non-null  object        
dtypes: datetime64[

# 2. Collection Analysis


## 2.1 Category Market Cap
The NFT market is broadly categorised into 14 types + Voulme + MarketCap + Liquidy

In [42]:
# Line chart - Volume - Sales
df_v = df.groupby(['BLOCK_TIMESTAMP','CATEGORY']).agg({'QUANTITY':'sum', 
                                                          'PRICE':'sum','PRICE_USD':'sum'}).reset_index()


df_v = df_v.rename(columns={
    'QUANTITY': 'Volume',
    'PRICE_USD': 'USD',
    'PRICE': 'ETH'})


df_v

,BLOCK_TIMESTAMP,CATEGORY,Volume,ETH,USD
0,2025-01-01 00:00:11,Other,1,0,9
1,2025-01-01 00:00:59,Other,1,0,33
2,2025-01-01 00:01:47,PFP,1,0,41
3,2025-01-01 00:02:35,PFP,1,0,365
4,2025-01-01 00:02:59,Other,1,0,135
...,...,...,...,...,...
231305,2025-04-27 11:56:35,Metaverse,1,0,0
231306,2025-04-27 11:56:35,Other,1,0,20
231307,2025-04-27 11:56:59,Other,1,0,0
231308,2025-04-27 11:57:59,Metaverse,1,0,0


In [43]:
df_v['BLOCK_TIMESTAMP'].max()

Timestamp('2025-04-27 11:58:11')

In [45]:
test = df[df['CATEGORY'] == 'Other']
test_2 = test.groupby(['PROJECT_NAME']).agg({'QUANTITY':'sum'}).sort_values(by='QUANTITY',ascending=False).head(40)
test_2

,QUANTITY
PROJECT_NAME,
Morph Black,783
goblintown,782
V,737
RiotBulls,731
Memes & Myths,722
SkyCastle Companions - Genesis,719
Peplicator,660
OKX AI NFT - ETH,628
Seeing Signs,627


In [16]:
df_v.to_csv('/Users/bahareh/Desktop/My_Job/BlochChain/Git/blockchain-dashboards/02_NFT/partt_2_collection_analysis.csv.csv')